In [3]:
import os

os.mkdir("data/step1_cov/")

In [5]:
!scp jbrown@eofe4.mit.edu:/nobackup1/jbrown/nahant_phage_reads/multi_contig_check/*genomecoverage data/step1_cov/

1.004.O.step1.clc.genomecoverage              100% 1931KB   8.7KB/s   03:41    
1.005.O.step1.clc.genomecoverage              100% 1853KB   7.9KB/s   03:55    
1.007.O.step1.clc.genomecoverage              100% 1800KB  12.1KB/s   02:29    
1.009.O.step1.clc.genomecoverage              100% 2503KB  25.0KB/s   01:40    
1.011.O.step1.clc.genomecoverage              100%  237KB  26.4KB/s   00:09    
1.012.O.step1.clc.genomecoverage              100% 2188KB  22.1KB/s   01:39    
1.013.O.step1.clc.genomecoverage              100% 2245KB  10.1KB/s   03:42    
1.016.O.step1.clc.genomecoverage              100% 2312KB  18.1KB/s   02:08    
1.017.O.step1.clc.genomecoverage              100% 1469KB  19.3KB/s   01:16    
1.020.O.step1.clc.genomecoverage              100% 2528KB  19.8KB/s   02:08    
1.021.A.step1.clc.genomecoverage              100%  933KB  17.9KB/s   00:52    
1.021.B.step1.clc.genomecoverage              100%  983KB  14.7KB/s   01:07    
1.021.C.step1.clc.genomecoverage        

In [5]:
class OnlineVariance(object):
    """
    Welford's algorithm computes the sample variance incrementally.
    from: http://stackoverflow.com/questions/5543651/computing-standard-deviation-in-a-stream
    
    Example use:
    > N = 100
    > data = np.random.random(N)
    > ov = OnlineVariance(ddof=0)
    > for d in data:
    >     ov.include(d)
    > std = ov.std
    > print(std)
    """

    def __init__(self, iterable=None, ddof=1):
        self.ddof, self.n, self.mean, self.M2 = ddof, 0, 0.0, 0.0
        if iterable is not None:
            for datum in iterable:
                self.include(datum)

    def include(self, datum):
        self.n += 1
        self.delta = datum - self.mean
        self.mean += self.delta / self.n
        self.M2 += self.delta * (datum - self.mean)
        self.variance = self.M2 / (self.n - self.ddof)

    @property
    def std(self):
        return np.sqrt(self.variance)

In [9]:
import pandas as pd
import glob
import os.path as op
import os
import numpy as np

In [2]:
def cov_tbl(bed_cov):
    '''
    create dict of mean coverage per contig based on bedtools coverage per base output
    '''
    coverage = pd.read_csv(bed_cov, sep="\t", header=None)
    contig_sizes = coverage.groupby([0])[1].max()
    mean_coverage = coverage.groupby([0])[2].mean()
    return contig_sizes, mean_coverage

In [19]:
if op.exists("./data/step1_cov/adjusted/")==False:
    os.mkdir("./data/step1_cov/adjusted/")
    
for l in glob.glob("./data/step1_cov/*genomecoverage")[:1]:
    name = op.basename(l).replace("step1.clc.genomecoverage","")
    coverage = pd.read_csv(l, sep="\t", header=None)
    contig_sizes = coverage.groupby([0])[1].max()
    mean_coverage = coverage.groupby([0])[2].mean()
    together = pd.concat([contig_sizes, mean_coverage], axis=1)
    together.rename(columns={'0': 'contig', '1': 'bp', '2':'coverage'}, inplace=True)
    together.columns = ['bp', 'coverage']
    
    
    names = []
    c1var = OnlineVariance(ddof=0)
    covar = OnlineVariance(ddof=0)
    coverage['is1'] = [i=i if i=="contig_1" for i in coverage[0] else i="contig_other"]
    for i, r in coverage.iterrows():

        if r[0] == "contig_1":
            c1_name = "contig_1"
            r[4] = "contig_1"
            c1var.include(r[2])
        else:
            co_name = "other_contigs"
            r[4] = "contig_other"
            covar.include(r[2])
        newdf.append(r)
    print(newdf)
    print(coverage)
    print(name, c1var.std, covar.std)
    #    df1 = pd.DataFrame({"contig_1":)


SyntaxError: invalid syntax (<ipython-input-19-84d101f9740b>, line 17)

In [16]:
print(newdf)

Empty DataFrame
Columns: []
Index: []


In [44]:
for l in glob.glob("./data/step1_cov/*genomecoverage")[:1]:
    name = op.basename(l).replace("step1.clc.genomecoverage","")
    coverage = pd.read_csv(l, sep="\t", header=None)
    contig_sizes = coverage.groupby([0])[1].max()
    mean_coverage = coverage.groupby([0])[2].mean()
    together = pd.concat([contig_sizes, mean_coverage], axis=1)
    together.columns = ['bp', 'coverage']
    c1 = together[together.index=='contig_1']
    c1['name'] = name
    others_bp = together[together.index != 'contig_1']['bp'].mean()
    others_cov = together[together.index != 'contig_1']['coverage'].mean()
    data = [others_bp, others_cov, name]
    df = pd.DataFrame(data, )

TypeError: cannot concatenate a non-NDFrame object

,bp,coverage
0,,
contig_1,45496,25813.746901


In [8]:
!open ./data/step1_cov/adjusted/*.genomecovadjust.txt -a textedit

In [13]:
%load_ext rpy2.ipython
%R library(ggplot2); library(dplyr)

//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
//anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)


array(['dplyr', 'ggplot2', 'tools', 'stats', 'graphics', 'grDevices',
       'utils', 'datasets', 'methods', 'base'], 
      dtype='|S9')

In [17]:
%%R
ctgct <- read.table("./data/asm_contig_cts.txt", sep="\t", header=TRUE)

head(ctgct)

     phage assembly1_contig_count
1 1.003.O.                      1
2 1.004.O.                     61
3 1.005.O.                     67
4 1.007.O.                     27
5 1.008.O.                      1
6 1.009.O.                     44


In [21]:
%%R
ctgct %>% nrow %>% print
ctgct %>% filter(assembly1_contig_count==1) %>% nrow %>% print

[1] 285
[1] 154
